In [2]:
#프로젝트의 루트 디렉토리를 설정하기 위한 코드
import sys
import os
current_dir = os.getcwd()  # 현재 작업 디렉토리
print(os.path.dirname(current_dir))
# print(os.path.dirname(os.path.dirname(current_dir)))
# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = r"c:\sb-fsts\sb-fsts"
sys.path.append(project_root)
os.chdir(project_root)  # 루트 경로로 변경

c:\sb-fsts\sb-fsts


In [3]:
import datetime
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
import numpy as np
import requests
from datetime import timedelta

In [10]:
#예시 데이터
class KisDomesticDailyChartBar:
    def __init__(self, time, open, close, high, low, volume, amount, change):
        self.time = time
        self.open = open
        self.close = close
        self.high = high
        self.low = low
        self.volume = volume
        self.amount = amount
        self.change = change


ohlc = [
    KisDomesticDailyChartBar("2023-01-01T00:00:00+09:00", 100000, 95000, 100000, 95000, 0, 0, 0),
    KisDomesticDailyChartBar("2023-01-02T00:00:00+09:00", 93000, 110000, 115000, 92000, 0, 0, 0),
    KisDomesticDailyChartBar("2023-01-03T00:00:00+09:00", 98000, 120000, 120000, 98000, 0, 0, 0),
    KisDomesticDailyChartBar("2023-01-04T00:00:00+09:00", 172500, 174000, 175000, 171000, 0, 0, 0),
]


In [7]:
from app.utils.stock_auto_trading import AutoTradingStock
from datetime import date, time


auto_trading_stock = AutoTradingStock()

symbol = '336260'

start_date = date(2023, 1, 1)
end_date = date(2023, 12, 1)

auto_trading_stock._get_ohlc(symbol, start_date, end_date)
ohlc = auto_trading_stock._get_ohlc(symbol, start_date, end_date)

[KisDomesticDailyChartBar(time='2022-01-03T00:00:00+09:00', open=96000, close=97600, high=97700, low=94900, volume=225810, amount=21764583400, change=2400),
 KisDomesticDailyChartBar(time='2022-01-04T00:00:00+09:00', open=97600, close=101500, high=102000, low=96400, volume=425113, amount=42465875100, change=3900),
 KisDomesticDailyChartBar(time='2022-01-05T00:00:00+09:00', open=101500, close=104000, high=105000, low=101500, volume=488265, amount=50636830000, change=2500),
 KisDomesticDailyChartBar(time='2022-01-06T00:00:00+09:00', open=104000, close=111000, high=112500, low=100500, volume=845951, amount=92124680000, change=7000),
 KisDomesticDailyChartBar(time='2022-01-07T00:00:00+09:00', open=111000, close=112000, high=114000, low=107000, volume=427073, amount=47353681000, change=1000),
 KisDomesticDailyChartBar(time='2022-01-10T00:00:00+09:00', open=112000, close=111500, high=116500, low=110500, volume=432107, amount=48718379500, change=-500),
 KisDomesticDailyChartBar(time='2022-01-

In [11]:
def trading_signal_penetrating(ohlc):
    """
    관통형
    매수/매도 신호를 판단하는 함수.
    :param ohlc: 날짜별 OHLC 데이터 (KisDomesticDailyChartBar 객체 리스트)
    :return: 매수 성공 목록과 개별 신호
    """
    results = []  # 전체 결과 저장
    successful_trades = []  # 매수 성공 목록

    if len(ohlc) < 3:
        raise ValueError("3일 이상의 데이터가 필요합니다.")

    for i in range(2, len(ohlc)):
        d_2 = ohlc[i - 2]
        d_1 = ohlc[i - 1]
        current = ohlc[i]

        # D-2 조건
        d_2_condition = d_2.close < d_2.open
        d_2_long_bear = abs(d_2.close - d_2.open) >= (float(d_2.open) * 0.03)
        print(f"\n[D-2 조건] 날짜: {d_2.time}")
        print(f"  종가 < 시초가: {d_2.close} < {d_2.open} = {d_2_condition}")
        print(f"  장대음봉: |{d_2.close} - {d_2.open}| >= {float(d_2.open) * 0.03} = {d_2_long_bear}")
        # D-1 조건
        d_1_condition = (
            d_1.open < d_2.low and
            d_1.close > d_2.close + (d_2.open - d_2.close) / 2
        )
        print(f"\n[D-1 조건] 날짜: {d_1.time}")
        print(f"  시초가 < D-2 최저가: {d_1.open} < {d_2.low} = {d_1.open < d_2.low}")
        
        # 모든 조건 충족 여부 확인
        all_conditions_met = d_2_condition and d_2_long_bear and d_1_condition

        if all_conditions_met:
            # 매매 시점 나누기
            buy_signal_d2 = current.close > d_2.high
            buy_signal_d1 = current.close > d_1.high
            print(f"\n[매매 시점] 날짜: {current.time}")
            print(f"  종가 > D-2 최고가: {current.close} > {d_2.high} = {buy_signal_d2}")
            print(f"  종가 > D-1 최고가: {current.close} > {d_1.high} = {buy_signal_d1}")
            
            if buy_signal_d2 or buy_signal_d1:
                # 매수 성공 여부
                buy_condition = "P > D-2.h" if buy_signal_d2 else "P > D-1.h"
                success = True  # 매수 성공

                # 결과 저장
                trade_result = {
                    "날짜": current.time,
                    "매수/매도": "매수신호",
                    "특징": "D-2 장대음봉",
                    "매매시점": buy_condition,
                    "손절조건": "미충족" if current.close >= d_1.low else "충족",
                    "매수 성공 여부": "성공" if success else "실패",
                }
                results.append(trade_result)

                # 매수 성공 목록에 추가
                successful_trades.append(trade_result)
            else:
                # 매수 신호는 있으나 성공하지 않음
                results.append({
                    "날짜": current.time,
                    "매수/매도": "매수신호",
                    "특징": "D-2 장대음봉",
                    "매매시점": "미충족",
                    "손절조건": "미충족",
                    "매수 성공 여부": "실패",
                })
        else:
            # 조건 충족하지 않음
            results.append({
                "날짜": current.time,
                "매수/매도": "조건 미충족",
                "특징": "미충족",
                "매매시점": "미충족",
                "손절조건": "미충족",
                "매수 성공 여부": "실패",
            })

    return results, successful_trades

# 매수/매도 신호 판단
signals, successful_trades = trading_signal_penetrating(ohlc)

# 전체 신호 출력
print("전체 신호:")
for signal in signals:
    print(signal)

# 매수 성공 목록 출력
print("\n매수 성공 목록:")
for trade in successful_trades:
    print(trade)



[D-2 조건] 날짜: 2023-01-01T00:00:00+09:00
  종가 < 시초가: 95000 < 100000 = True
  장대음봉: |95000 - 100000| >= 3000.0 = True

[D-1 조건] 날짜: 2023-01-02T00:00:00+09:00
  시초가 < D-2 최저가: 93000 < 95000 = True

[매매 시점] 날짜: 2023-01-03T00:00:00+09:00
  종가 > D-2 최고가: 120000 > 100000 = True
  종가 > D-1 최고가: 120000 > 115000 = True

[D-2 조건] 날짜: 2023-01-02T00:00:00+09:00
  종가 < 시초가: 110000 < 93000 = False
  장대음봉: |110000 - 93000| >= 2790.0 = True

[D-1 조건] 날짜: 2023-01-03T00:00:00+09:00
  시초가 < D-2 최저가: 98000 < 92000 = False
전체 신호:
{'날짜': '2023-01-03T00:00:00+09:00', '매수/매도': '매수신호', '특징': 'D-2 장대음봉', '매매시점': 'P > D-2.h', '손절조건': '미충족', '매수 성공 여부': '성공'}
{'날짜': '2023-01-04T00:00:00+09:00', '매수/매도': '조건 미충족', '특징': '미충족', '매매시점': '미충족', '손절조건': '미충족', '매수 성공 여부': '실패'}

매수 성공 목록:
{'날짜': '2023-01-03T00:00:00+09:00', '매수/매도': '매수신호', '특징': 'D-2 장대음봉', '매매시점': 'P > D-2.h', '손절조건': '미충족', '매수 성공 여부': '성공'}


In [ ]:
trading_signal_penetrating(ohlc)
successful_trades

In [12]:
def trading_signal_engulfing(ohlc):
    """
    상승장악형1 매수/매도 신호를 판단하는 함수.
    :param ohlc: 날짜별 OHLC 데이터 (KisDomesticDailyChartBar 객체 리스트)
    :return: 매수 성공 목록과 개별 신호
    """
    results = []  # 전체 결과 저장
    successful_trades = []  # 매수 성공 목록

    if len(ohlc) < 3:
        raise ValueError("3일 이상의 데이터가 필요합니다.")

    for i in range(2, len(ohlc)):
        d_2 = ohlc[i - 2]  # D-2 데이터
        d_1 = ohlc[i - 1]  # D-1 데이터
        current = ohlc[i]  # 당일 데이터

        # D-2 조건 디버깅 (음봉)
        d_2_condition = d_2.close < d_2.open
        print(f"\n[D-2 조건] 날짜: {d_2.time}")
        print(f"  종가 < 시초가: {d_2.close} < {d_2.open} = {d_2_condition}")

        # D-1 조건 디버깅
        d_1_condition = (
            d_1.open < d_2.low and
            d_1.close > d_2.high
        )
        print(f"\n[D-1 조건] 날짜: {d_1.time}")
        print(f"  시초가 < D-2 최저가: {d_1.open} < {d_2.low} = {d_1.open < d_2.low}")
        print(f"  종가 > D-2 최고가: {d_1.close} > {d_2.high} = {d_1.close > d_2.high}")

        # 매매 시점 디버깅
        buy_signal = current.close > d_1.high
        print(f"\n[매매 시점] 날짜: {current.time}")
        print(f"  종가 > D-1 최고가: {current.close} > {d_1.high} = {buy_signal}")

        # 손절 조건 디버깅
        stoploss = current.close < d_1.low
        print(f"\n[손절 조건] 날짜: {current.time}")
        print(f"  종가 < D-1 최저가: {current.close} < {d_1.low} = {stoploss}")

        # 모든 조건 충족 여부 확인
        all_conditions_met = d_2_condition and d_1_condition
        print(f"\n[전체 조건 충족 여부] 날짜: {current.time}")
        print(f"  D-2 조건: {d_2_condition}")
        print(f"  D-1 조건: {d_1_condition}")
        print(f"  모든 조건 충족: {all_conditions_met}")

        if all_conditions_met:
            if buy_signal:
                trade_result = {
                    "날짜": current.time,
                    "매수/매도": "매수신호",
                    "특징": "상승장악형1",
                    "매매시점": "P > H",
                    "손절조건": "미충족" if not stoploss else "충족",
                    "매수 성공 여부": "성공",
                }
                results.append(trade_result)
                successful_trades.append(trade_result)
            else:
                results.append({
                    "날짜": current.time,
                    "매수/매도": "매수신호",
                    "특징": "상승장악형1",
                    "매매시점": "미충족",
                    "손절조건": "미충족" if not stoploss else "충족",
                    "매수 성공 여부": "실패",
                })
        else:
            results.append({
                "날짜": current.time,
                "매수/매도": "조건 미충족",
                "특징": "미충족",
                "매매시점": "미충족",
                "손절조건": "미충족",
                "매수 성공 여부": "실패",
            })

    return results, successful_trades

# 매수/매도 신호 판단
signals, successful_trades = trading_signal_engulfing(ohlc)

# 전체 신호 출력
print("\n전체 신호:")
for signal in signals:
    print(signal)

# 매수 성공 목록 출력
print("\n매수 성공 목록:")
for trade in successful_trades:
    print(trade)



[D-2 조건] 날짜: 2023-01-01T00:00:00+09:00
  종가 < 시초가: 95000 < 100000 = True

[D-1 조건] 날짜: 2023-01-02T00:00:00+09:00
  시초가 < D-2 최저가: 93000 < 95000 = True
  종가 > D-2 최고가: 110000 > 100000 = True

[매매 시점] 날짜: 2023-01-03T00:00:00+09:00
  종가 > D-1 최고가: 120000 > 115000 = True

[손절 조건] 날짜: 2023-01-03T00:00:00+09:00
  종가 < D-1 최저가: 120000 < 92000 = False

[전체 조건 충족 여부] 날짜: 2023-01-03T00:00:00+09:00
  D-2 조건: True
  D-1 조건: True
  모든 조건 충족: True

[D-2 조건] 날짜: 2023-01-02T00:00:00+09:00
  종가 < 시초가: 110000 < 93000 = False

[D-1 조건] 날짜: 2023-01-03T00:00:00+09:00
  시초가 < D-2 최저가: 98000 < 92000 = False
  종가 > D-2 최고가: 120000 > 115000 = True

[매매 시점] 날짜: 2023-01-04T00:00:00+09:00
  종가 > D-1 최고가: 174000 > 120000 = True

[손절 조건] 날짜: 2023-01-04T00:00:00+09:00
  종가 < D-1 최저가: 174000 < 98000 = False

[전체 조건 충족 여부] 날짜: 2023-01-04T00:00:00+09:00
  D-2 조건: False
  D-1 조건: False
  모든 조건 충족: False

전체 신호:
{'날짜': '2023-01-03T00:00:00+09:00', '매수/매도': '매수신호', '특징': '상승장악형1', '매매시점': 'P > H', '손절조건': '미충족', '매수 성공 여